# 59. SfM パイプライン基礎
## Structure from Motion Pipeline Basics

---

## 学習目標

このノートブックを完了すると、以下ができるようになります：

- [ ] Structure from Motion (SfM) の全体的なワークフローを理解する
- [ ] 2視点からの初期化（エッセンシャル行列→カメラ姿勢→3D点）を実装できる
- [ ] インクリメンタル SfM の概念を理解する
- [ ] PnP 問題によるカメラ姿勢推定を理解する
- [ ] トラック（マルチビュー対応）の概念を理解する
- [ ] SfM システムの精度評価方法を理解する

---

## 前提知識

- 55: エピポーラ幾何の理論
- 57: 三角測量と3D復元
- 58: 特徴点検出とマッチング

**難易度**: ★★★★★（上級）  
**推定学習時間**: 120-150分

---

## 1. Structure from Motion (SfM) とは

**Structure from Motion (SfM)** は、複数の2D画像から3D構造（点群）とカメラ姿勢を同時に復元する技術です。

### 入力と出力

| 入力 | 出力 |
|------|------|
| 複数の2D画像 | 3D点群（Structure） |
| カメラ内部パラメータ（既知または推定） | カメラ姿勢（Motion） |

### SfM の種類

| 種類 | 説明 | 用途 |
|------|------|------|
| **インクリメンタル SfM** | 画像を1枚ずつ追加 | COLMAP, VisualSFM |
| **グローバル SfM** | 全画像を同時に処理 | 1DSfM, Theia |
| **ハイブリッド** | 両方の組み合わせ | OpenMVG |

### 応用分野

- 3Dスキャン・モデリング
- 自動運転（視覚オドメトリ）
- AR/VR
- ドローン測量
- NeRF のカメラ姿勢推定

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from typing import Tuple, List, Dict, Optional
from scipy.optimize import least_squares
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

np.set_printoptions(precision=4, suppress=True)

print("ライブラリのインポート完了")

---

## 2. インクリメンタル SfM パイプライン

### 2.1 全体フロー

```
1. 特徴点検出・マッチング（全画像ペア）
       ↓
2. 初期化（最良の2画像ペアを選択）
   - エッセンシャル行列の推定
   - カメラ姿勢の復元
   - 初期3D点の三角測量
       ↓
3. 画像の追加（繰り返し）
   - PnP でカメラ姿勢を推定
   - 新しい3D点を三角測量
       ↓
4. バンドル調整（最適化）
       ↓
5. 点群とカメラ姿勢の出力
```

In [ ]:
def visualize_sfm_pipeline():
    """SfM パイプラインの概念図"""
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    
    steps = [
        ('1. Feature Detection & Matching', 'Multiple images → Keypoints & Matches'),
        ('2. Initialization (2-view)', 'Essential matrix → R, t → Triangulation'),
        ('3. Add New Image (PnP)', '2D-3D correspondences → Camera pose'),
        ('4. Triangulate New Points', 'New matches → New 3D points'),
        ('5. Bundle Adjustment', 'Optimize cameras & points jointly'),
        ('6. Output', '3D point cloud + Camera poses')
    ]
    
    for ax, (title, desc) in zip(axes.flat, steps):
        ax.text(0.5, 0.6, title, ha='center', va='center', fontsize=12, fontweight='bold')
        ax.text(0.5, 0.4, desc, ha='center', va='center', fontsize=10, style='italic')
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        ax.axis('off')
        ax.add_patch(plt.Rectangle((0.05, 0.2), 0.9, 0.6, fill=False, edgecolor='blue', linewidth=2))
    
    plt.suptitle('Incremental SfM Pipeline', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

visualize_sfm_pipeline()

---

## 3. 合成データの準備

SfM パイプラインをテストするための合成シーンを作成します。

In [ ]:
class SyntheticScene:
    """合成3Dシーン（SfMテスト用）"""
    
    def __init__(self, n_points: int = 100, n_cameras: int = 5):
        self.n_points = n_points
        self.n_cameras = n_cameras
        
        # カメラ内部パラメータ
        self.K = np.array([
            [500, 0, 320],
            [0, 500, 240],
            [0, 0, 1]
        ])
        
        self.image_size = (640, 480)
        
        # 3D点を生成
        self._generate_points()
        
        # カメラを配置
        self._generate_cameras()
        
        # 投影を計算
        self._project_points()
    
    def _generate_points(self):
        """ランダムな3D点を生成"""
        np.random.seed(42)
        self.points_3d = np.random.randn(self.n_points, 3) * 2
        self.points_3d[:, 2] = np.abs(self.points_3d[:, 2]) + 3  # Z > 3
    
    def _generate_cameras(self):
        """円周上にカメラを配置"""
        self.cameras = []  # [(R, t), ...]
        radius = 8
        
        for i in range(self.n_cameras):
            angle = 2 * np.pi * i / self.n_cameras - np.pi / 2
            
            # カメラ位置（円周上）
            C = np.array([radius * np.cos(angle), 0, radius * np.sin(angle)])
            
            # 原点を向くように回転
            z_axis = -C / np.linalg.norm(C)
            y_axis = np.array([0, 1, 0])
            x_axis = np.cross(y_axis, z_axis)
            x_axis = x_axis / np.linalg.norm(x_axis)
            y_axis = np.cross(z_axis, x_axis)
            
            R = np.vstack([x_axis, y_axis, z_axis])
            t = -R @ C
            
            self.cameras.append((R, t))
    
    def _project_points(self):
        """3D点を各カメラに投影"""
        self.projections = []  # [[(u, v), ...], ...] for each camera
        self.visibility = []  # [[True/False, ...], ...] for each camera
        
        for R, t in self.cameras:
            P = self.K @ np.hstack([R, t.reshape(-1, 1)])
            
            cam_projs = []
            cam_vis = []
            
            for X in self.points_3d:
                X_h = np.append(X, 1)
                x_h = P @ X_h
                x = x_h[:2] / x_h[2]
                
                # 可視性チェック
                visible = (0 <= x[0] < self.image_size[0] and 
                           0 <= x[1] < self.image_size[1] and
                           x_h[2] > 0)  # 前方にある
                
                cam_projs.append(x)
                cam_vis.append(visible)
            
            self.projections.append(np.array(cam_projs))
            self.visibility.append(np.array(cam_vis))
    
    def get_projection_matrix(self, cam_idx: int) -> np.ndarray:
        """指定カメラの投影行列を取得"""
        R, t = self.cameras[cam_idx]
        return self.K @ np.hstack([R, t.reshape(-1, 1)])
    
    def get_visible_points(self, cam_idx: int) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
        """指定カメラで見える点を取得"""
        vis = self.visibility[cam_idx]
        return (self.projections[cam_idx][vis], 
                self.points_3d[vis],
                np.where(vis)[0])
    
    def add_noise(self, noise_level: float = 1.0):
        """投影点にノイズを追加"""
        for i in range(len(self.projections)):
            noise = np.random.randn(*self.projections[i].shape) * noise_level
            self.projections[i] = self.projections[i] + noise

# シーンの作成
scene = SyntheticScene(n_points=80, n_cameras=6)
scene.add_noise(noise_level=0.5)

print(f"3D点数: {scene.n_points}")
print(f"カメラ数: {scene.n_cameras}")

for i in range(scene.n_cameras):
    visible_count = np.sum(scene.visibility[i])
    print(f"  カメラ{i}: {visible_count} 点が可視")

In [ ]:
def visualize_scene(scene: SyntheticScene):
    """合成シーンの可視化"""
    fig = plt.figure(figsize=(14, 6))
    
    # 3Dビュー
    ax1 = fig.add_subplot(121, projection='3d')
    
    # 3D点
    ax1.scatter(scene.points_3d[:, 0], scene.points_3d[:, 1], scene.points_3d[:, 2],
                c='blue', s=20, alpha=0.5, label='3D Points')
    
    # カメラ
    colors = plt.cm.tab10(np.linspace(0, 1, scene.n_cameras))
    for i, (R, t) in enumerate(scene.cameras):
        C = -R.T @ t  # カメラ中心
        ax1.scatter(*C, color=colors[i], s=200, marker='o', label=f'Camera {i}')
        
        # カメラの向き
        direction = R.T @ np.array([0, 0, 1])  # Z軸方向
        ax1.quiver(*C, *direction * 1.5, color=colors[i], arrow_length_ratio=0.2)
    
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')
    ax1.set_zlabel('Z')
    ax1.set_title('Synthetic Scene (Ground Truth)')
    ax1.view_init(elev=30, azim=45)
    
    # 画像投影（最初のカメラ）
    ax2 = fig.add_subplot(122)
    
    pts, _, _ = scene.get_visible_points(0)
    ax2.scatter(pts[:, 0], pts[:, 1], c='red', s=10)
    ax2.set_xlim(0, scene.image_size[0])
    ax2.set_ylim(scene.image_size[1], 0)
    ax2.set_aspect('equal')
    ax2.set_title('Camera 0 Projection (with noise)')
    ax2.set_xlabel('u (pixels)')
    ax2.set_ylabel('v (pixels)')
    
    plt.tight_layout()
    plt.show()

visualize_scene(scene)

---

## 4. 2視点初期化

### 4.1 初期ペアの選択

良い初期ペアの条件：
- 十分なマッチ数
- 適度な視差（ベースラインが狭すぎない）
- 高いインライア率

### 4.2 処理フロー

1. 本質行列 $\mathbf{E}$ の推定（5点法 or 8点法 + RANSAC）
2. $\mathbf{E}$ から $(\mathbf{R}, \mathbf{t})$ を復元
3. 三角測量で初期3D点を復元

In [ ]:
def normalize_points(pts: np.ndarray, K: np.ndarray) -> np.ndarray:
    """画像座標を正規化座標に変換"""
    pts_h = np.hstack([pts, np.ones((len(pts), 1))])
    pts_norm = (np.linalg.inv(K) @ pts_h.T).T
    return pts_norm[:, :2]

def eight_point_algorithm(pts1: np.ndarray, pts2: np.ndarray) -> np.ndarray:
    """8点法による本質行列の推定（正規化座標）"""
    n = len(pts1)
    A = np.zeros((n, 9))
    
    for i in range(n):
        x1, y1 = pts1[i]
        x2, y2 = pts2[i]
        A[i] = [x2*x1, x2*y1, x2, y2*x1, y2*y1, y2, x1, y1, 1]
    
    _, _, Vt = np.linalg.svd(A)
    E = Vt[-1].reshape(3, 3)
    
    # ランク2に射影
    U, S, Vt = np.linalg.svd(E)
    S = np.array([1, 1, 0])  # 特異値を正規化
    E = U @ np.diag(S) @ Vt
    
    return E

def decompose_essential_matrix(E: np.ndarray) -> List[Tuple[np.ndarray, np.ndarray]]:
    """本質行列を分解して4つの解候補を返す"""
    U, _, Vt = np.linalg.svd(E)
    
    if np.linalg.det(U) < 0:
        U = -U
    if np.linalg.det(Vt) < 0:
        Vt = -Vt
    
    W = np.array([[0, -1, 0], [1, 0, 0], [0, 0, 1]])
    
    R1 = U @ W @ Vt
    R2 = U @ W.T @ Vt
    t = U[:, 2]
    
    # det(R) = 1 を保証
    if np.linalg.det(R1) < 0:
        R1 = -R1
    if np.linalg.det(R2) < 0:
        R2 = -R2
    
    return [(R1, t), (R1, -t), (R2, t), (R2, -t)]

def triangulate_dlt(P1: np.ndarray, P2: np.ndarray, 
                    x1: np.ndarray, x2: np.ndarray) -> np.ndarray:
    """DLTによる三角測量"""
    A = np.array([
        x1[0] * P1[2] - P1[0],
        x1[1] * P1[2] - P1[1],
        x2[0] * P2[2] - P2[0],
        x2[1] * P2[2] - P2[1]
    ])
    
    _, _, Vt = np.linalg.svd(A)
    X_h = Vt[-1]
    return X_h[:3] / X_h[3]

def check_cheirality(R: np.ndarray, t: np.ndarray, K: np.ndarray,
                     pts1: np.ndarray, pts2: np.ndarray) -> int:
    """チェイラリティ条件をチェック（両カメラ前方にある点の数）"""
    P1 = K @ np.hstack([np.eye(3), np.zeros((3, 1))])
    P2 = K @ np.hstack([R, t.reshape(-1, 1)])
    
    n_valid = 0
    for i in range(min(len(pts1), 50)):
        X = triangulate_dlt(P1, P2, pts1[i], pts2[i])
        
        # カメラ1での深度
        z1 = X[2]
        
        # カメラ2での深度
        X_cam2 = R @ X + t
        z2 = X_cam2[2]
        
        if z1 > 0 and z2 > 0:
            n_valid += 1
    
    return n_valid

def recover_pose(E: np.ndarray, K: np.ndarray, 
                 pts1: np.ndarray, pts2: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """本質行列からカメラ姿勢を復元"""
    solutions = decompose_essential_matrix(E)
    
    best_R, best_t = None, None
    best_count = 0
    
    for R, t in solutions:
        count = check_cheirality(R, t, K, pts1, pts2)
        if count > best_count:
            best_count = count
            best_R, best_t = R, t
    
    return best_R, best_t

print("2視点初期化関数の実装完了")

In [ ]:
def initialize_two_view(scene: SyntheticScene, 
                        cam1_idx: int = 0, 
                        cam2_idx: int = 1) -> Dict:
    """2視点からSfMを初期化"""
    K = scene.K
    
    # 両方のカメラで見える点を取得
    vis1 = scene.visibility[cam1_idx]
    vis2 = scene.visibility[cam2_idx]
    common_vis = vis1 & vis2
    
    pts1 = scene.projections[cam1_idx][common_vis]
    pts2 = scene.projections[cam2_idx][common_vis]
    pts_3d_true = scene.points_3d[common_vis]
    point_indices = np.where(common_vis)[0]
    
    print(f"共通可視点: {len(pts1)}")
    
    # 正規化座標に変換
    pts1_norm = normalize_points(pts1, K)
    pts2_norm = normalize_points(pts2, K)
    
    # 本質行列の推定
    E = eight_point_algorithm(pts1_norm, pts2_norm)
    
    # カメラ姿勢の復元
    R, t = recover_pose(E, K, pts1, pts2)
    
    # 三角測量
    P1 = K @ np.hstack([np.eye(3), np.zeros((3, 1))])
    P2 = K @ np.hstack([R, t.reshape(-1, 1)])
    
    points_3d_estimated = []
    for i in range(len(pts1)):
        X = triangulate_dlt(P1, P2, pts1[i], pts2[i])
        points_3d_estimated.append(X)
    
    points_3d_estimated = np.array(points_3d_estimated)
    
    # スケール補正（真値との比較用）
    # 真のスケールを適用
    R_true1, t_true1 = scene.cameras[cam1_idx]
    R_true2, t_true2 = scene.cameras[cam2_idx]
    
    # カメラ間の相対姿勢（真値）
    R_rel_true = R_true2 @ R_true1.T
    t_rel_true = t_true2 - R_rel_true @ t_true1
    scale_true = np.linalg.norm(t_rel_true)
    
    # 推定されたスケールを真値に合わせる
    t = t * scale_true
    points_3d_estimated = points_3d_estimated * scale_true
    
    result = {
        'cameras': {
            cam1_idx: (np.eye(3), np.zeros(3)),
            cam2_idx: (R, t)
        },
        'points_3d': points_3d_estimated,
        'point_indices': point_indices,
        'observations': {
            cam1_idx: pts1,
            cam2_idx: pts2
        }
    }
    
    return result

# 初期化の実行
sfm_result = initialize_two_view(scene, cam1_idx=0, cam2_idx=1)

print(f"\n初期化されたカメラ数: {len(sfm_result['cameras'])}")
print(f"復元された3D点数: {len(sfm_result['points_3d'])}")

In [ ]:
def evaluate_reconstruction(scene: SyntheticScene, sfm_result: Dict):
    """復元結果の評価"""
    # カメラ姿勢の誤差
    print("=== カメラ姿勢の評価 ===")
    for cam_idx, (R_est, t_est) in sfm_result['cameras'].items():
        R_true, t_true = scene.cameras[cam_idx]
        
        # カメラ0を基準とした相対姿勢で比較
        if cam_idx == 0:
            continue
        
        R_true0, t_true0 = scene.cameras[0]
        R_rel_true = R_true @ R_true0.T
        t_rel_true = t_true - R_rel_true @ t_true0
        
        # 回転誤差
        R_error = R_est @ R_rel_true.T
        angle_error = np.arccos(np.clip((np.trace(R_error) - 1) / 2, -1, 1))
        
        # 並進方向誤差
        t_est_norm = t_est / np.linalg.norm(t_est)
        t_true_norm = t_rel_true / np.linalg.norm(t_rel_true)
        t_angle_error = np.arccos(np.clip(np.abs(np.dot(t_est_norm, t_true_norm)), -1, 1))
        
        print(f"Camera {cam_idx}:")
        print(f"  回転誤差: {np.degrees(angle_error):.2f}°")
        print(f"  並進方向誤差: {np.degrees(t_angle_error):.2f}°")
    
    # 3D点の誤差
    print("\n=== 3D点の評価 ===")
    
    # 座標系の位置合わせ（Procrustes解析）
    pts_est = sfm_result['points_3d']
    pts_true = scene.points_3d[sfm_result['point_indices']]
    
    # カメラ0を原点とした座標系に変換
    R0, t0 = scene.cameras[0]
    pts_true_cam0 = (R0 @ pts_true.T).T + t0
    
    # 誤差計算
    errors = np.linalg.norm(pts_est - pts_true_cam0, axis=1)
    
    print(f"3D点数: {len(pts_est)}")
    print(f"平均誤差: {np.mean(errors):.4f}")
    print(f"中央値誤差: {np.median(errors):.4f}")
    print(f"最大誤差: {np.max(errors):.4f}")
    
    return errors

errors = evaluate_reconstruction(scene, sfm_result)

---

## 5. PnP (Perspective-n-Point) 問題

### 5.1 問題設定

既知の3D点と対応する2D画像点から、カメラ姿勢を推定します。

- **入力**: $n$ 組の 3D-2D 対応 $(\mathbf{X}_i, \mathbf{x}_i)$
- **出力**: カメラ姿勢 $(\mathbf{R}, \mathbf{t})$

### 5.2 最小ケース

- **P3P**: 3点で最大4解
- **P4P/P5P/P6P**: より多くの点でより安定

### 5.3 EPnP アルゴリズム

効率的な PnP ソルバー。$n \geq 4$ 点から線形に解を求める。

In [ ]:
def pnp_dlt(points_3d: np.ndarray, points_2d: np.ndarray, 
            K: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """DLT-based PnP (簡略版)
    
    6点以上から投影行列を推定し、R, t を分解
    """
    n = len(points_3d)
    assert n >= 6, "6点以上必要です"
    
    # 行列Aの構築
    A = np.zeros((2 * n, 12))
    
    for i in range(n):
        X, Y, Z = points_3d[i]
        u, v = points_2d[i]
        
        A[2*i] = [-X, -Y, -Z, -1, 0, 0, 0, 0, u*X, u*Y, u*Z, u]
        A[2*i+1] = [0, 0, 0, 0, -X, -Y, -Z, -1, v*X, v*Y, v*Z, v]
    
    # SVDで解く
    _, _, Vt = np.linalg.svd(A)
    P = Vt[-1].reshape(3, 4)
    
    # P = K [R | t] から R, t を分解
    M = np.linalg.inv(K) @ P[:, :3]
    
    # Mの特異値分解でRを抽出
    U, S, Vt = np.linalg.svd(M)
    R = U @ Vt
    
    # det(R) = 1 を保証
    if np.linalg.det(R) < 0:
        R = -R
    
    # スケールを復元
    scale = np.mean(S)
    t = np.linalg.inv(K) @ P[:, 3] / scale
    
    return R, t

def pnp_ransac(points_3d: np.ndarray, points_2d: np.ndarray,
               K: np.ndarray, threshold: float = 5.0,
               max_iterations: int = 1000) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """RANSAC による PnP
    
    Returns:
        R, t: カメラ姿勢
        inlier_mask: インライアのマスク
    """
    n = len(points_3d)
    best_R, best_t = None, None
    best_inliers = np.zeros(n, dtype=bool)
    best_n_inliers = 0
    
    for _ in range(max_iterations):
        # ランダムに6点選択
        indices = np.random.choice(n, 6, replace=False)
        
        try:
            R, t = pnp_dlt(points_3d[indices], points_2d[indices], K)
        except:
            continue
        
        # 再投影誤差を計算
        P = K @ np.hstack([R, t.reshape(-1, 1)])
        projected = []
        for X in points_3d:
            X_h = np.append(X, 1)
            x_h = P @ X_h
            projected.append(x_h[:2] / x_h[2])
        projected = np.array(projected)
        
        errors = np.linalg.norm(projected - points_2d, axis=1)
        inliers = errors < threshold
        n_inliers = np.sum(inliers)
        
        if n_inliers > best_n_inliers:
            best_n_inliers = n_inliers
            best_R, best_t = R, t
            best_inliers = inliers
    
    # インライアで再推定
    if np.sum(best_inliers) >= 6:
        best_R, best_t = pnp_dlt(points_3d[best_inliers], 
                                  points_2d[best_inliers], K)
    
    return best_R, best_t, best_inliers

print("PnP の実装完了")

---

## 6. インクリメンタル SfM

### 6.1 画像追加のフロー

1. 新しい画像と既存の3D点のマッチングを見つける
2. PnP でカメラ姿勢を推定
3. 新しい3D点を三角測量
4. バンドル調整で最適化

In [ ]:
def add_camera_to_sfm(scene: SyntheticScene, sfm_result: Dict, 
                      new_cam_idx: int) -> Dict:
    """新しいカメラをSfM結果に追加"""
    K = scene.K
    
    # 既存の3D点と新しいカメラでの観測点を対応付け
    existing_point_indices = sfm_result['point_indices']
    new_visibility = scene.visibility[new_cam_idx]
    
    # 共通点を見つける
    common_indices = []
    for i, pt_idx in enumerate(existing_point_indices):
        if new_visibility[pt_idx]:
            common_indices.append(i)
    
    common_indices = np.array(common_indices)
    
    if len(common_indices) < 6:
        print(f"共通点が不足: {len(common_indices)} < 6")
        return sfm_result
    
    # 3D点と2D観測点
    points_3d = sfm_result['points_3d'][common_indices]
    points_2d = scene.projections[new_cam_idx][existing_point_indices[common_indices]]
    
    print(f"カメラ {new_cam_idx} 追加: 共通点 {len(common_indices)}")
    
    # PnP でカメラ姿勢を推定
    R, t, inliers = pnp_ransac(points_3d, points_2d, K, threshold=5.0)
    
    print(f"  PnP インライア: {np.sum(inliers)} / {len(inliers)}")
    
    # 結果を更新
    sfm_result['cameras'][new_cam_idx] = (R, t)
    sfm_result['observations'][new_cam_idx] = points_2d
    
    # 新しい3D点を三角測量（既存カメラとの共通点で未復元のもの）
    new_points_3d = []
    new_point_indices = []
    
    for exist_cam_idx, (R_exist, t_exist) in sfm_result['cameras'].items():
        if exist_cam_idx == new_cam_idx:
            continue
        
        vis_exist = scene.visibility[exist_cam_idx]
        vis_new = scene.visibility[new_cam_idx]
        
        # 両方で見えるが未復元の点
        for pt_idx in range(scene.n_points):
            if vis_exist[pt_idx] and vis_new[pt_idx]:
                if pt_idx not in sfm_result['point_indices']:
                    # 三角測量
                    P_exist = K @ np.hstack([R_exist, t_exist.reshape(-1, 1)])
                    P_new = K @ np.hstack([R, t.reshape(-1, 1)])
                    
                    x_exist = scene.projections[exist_cam_idx][pt_idx]
                    x_new = scene.projections[new_cam_idx][pt_idx]
                    
                    X = triangulate_dlt(P_exist, P_new, x_exist, x_new)
                    
                    # 深度チェック
                    if X[2] > 0:
                        new_points_3d.append(X)
                        new_point_indices.append(pt_idx)
    
    # 新しい点を追加
    if len(new_points_3d) > 0:
        sfm_result['points_3d'] = np.vstack([sfm_result['points_3d'], 
                                              np.array(new_points_3d)])
        sfm_result['point_indices'] = np.concatenate([sfm_result['point_indices'],
                                                       np.array(new_point_indices)])
        print(f"  新しい3D点: {len(new_points_3d)}")
    
    return sfm_result

# 残りのカメラを追加
print("\n=== インクリメンタルSfM ===")
for cam_idx in range(2, scene.n_cameras):
    sfm_result = add_camera_to_sfm(scene, sfm_result, cam_idx)

print(f"\n最終結果:")
print(f"  カメラ数: {len(sfm_result['cameras'])}")
print(f"  3D点数: {len(sfm_result['points_3d'])}")

In [ ]:
def visualize_sfm_result(scene: SyntheticScene, sfm_result: Dict):
    """SfM結果の可視化"""
    fig = plt.figure(figsize=(14, 6))
    
    # 真値
    ax1 = fig.add_subplot(121, projection='3d')
    
    ax1.scatter(scene.points_3d[:, 0], scene.points_3d[:, 1], scene.points_3d[:, 2],
                c='blue', s=10, alpha=0.3, label='GT Points')
    
    colors = plt.cm.tab10(np.linspace(0, 1, scene.n_cameras))
    for i, (R, t) in enumerate(scene.cameras):
        C = -R.T @ t
        ax1.scatter(*C, color=colors[i], s=100, marker='o')
        direction = R.T @ np.array([0, 0, 1])
        ax1.quiver(*C, *direction, color=colors[i], arrow_length_ratio=0.3)
    
    ax1.set_title('Ground Truth')
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')
    ax1.set_zlabel('Z')
    ax1.view_init(elev=30, azim=45)
    
    # 推定結果
    ax2 = fig.add_subplot(122, projection='3d')
    
    ax2.scatter(sfm_result['points_3d'][:, 0], 
                sfm_result['points_3d'][:, 1], 
                sfm_result['points_3d'][:, 2],
                c='red', s=10, alpha=0.3, label='Estimated Points')
    
    for cam_idx, (R, t) in sfm_result['cameras'].items():
        C = -R.T @ t
        ax2.scatter(*C, color=colors[cam_idx], s=100, marker='o', 
                    label=f'Camera {cam_idx}')
        direction = R.T @ np.array([0, 0, 1])
        ax2.quiver(*C, *direction, color=colors[cam_idx], arrow_length_ratio=0.3)
    
    ax2.set_title('SfM Reconstruction')
    ax2.set_xlabel('X')
    ax2.set_ylabel('Y')
    ax2.set_zlabel('Z')
    ax2.view_init(elev=30, azim=45)
    
    plt.tight_layout()
    plt.show()

visualize_sfm_result(scene, sfm_result)

---

## 7. トラック（マルチビュー対応）

### 7.1 トラックとは

**トラック**は、同じ3D点に対応する複数の画像での2D観測点の集合です。

```
Track:
  3D Point X_i
    ├── Image 0: (u0, v0)
    ├── Image 1: (u1, v1)
    ├── Image 3: (u3, v3)
    └── Image 5: (u5, v5)
```

### 7.2 トラックの構築

1. 全画像ペアで特徴点マッチング
2. 推移的なマッチング（A-B, B-C → A-C）を構築
3. 矛盾するマッチを除去

In [ ]:
class Track:
    """特徴点のトラック（マルチビュー対応）"""
    
    def __init__(self, track_id: int):
        self.track_id = track_id
        self.observations = {}  # {cam_idx: (u, v), ...}
        self.point_3d = None
    
    def add_observation(self, cam_idx: int, uv: Tuple[float, float]):
        self.observations[cam_idx] = uv
    
    def get_camera_indices(self) -> List[int]:
        return list(self.observations.keys())
    
    def __len__(self) -> int:
        return len(self.observations)

def build_tracks(scene: SyntheticScene) -> List[Track]:
    """合成シーンからトラックを構築"""
    tracks = []
    
    for pt_idx in range(scene.n_points):
        track = Track(pt_idx)
        
        for cam_idx in range(scene.n_cameras):
            if scene.visibility[cam_idx][pt_idx]:
                uv = scene.projections[cam_idx][pt_idx]
                track.add_observation(cam_idx, tuple(uv))
        
        if len(track) >= 2:  # 少なくとも2視点で見える
            tracks.append(track)
    
    return tracks

# トラックの構築と統計
tracks = build_tracks(scene)

print(f"トラック数: {len(tracks)}")

track_lengths = [len(t) for t in tracks]
print(f"トラック長の分布:")
for length in range(2, max(track_lengths) + 1):
    count = track_lengths.count(length)
    if count > 0:
        print(f"  {length}視点: {count} トラック")

---

## 8. 再投影誤差の計算

### 8.1 定義

3D点 $\mathbf{X}$ をカメラ $i$ に投影した点と、実際の観測点との差：

$$e_i = \|\mathbf{x}_i - \pi(\mathbf{P}_i, \mathbf{X})\|$$

### 8.2 全体の誤差

$$E = \sum_i \sum_j \|\mathbf{x}_{ij} - \pi(\mathbf{P}_i, \mathbf{X}_j)\|^2$$

In [ ]:
def compute_reprojection_error(sfm_result: Dict, scene: SyntheticScene) -> Dict:
    """再投影誤差を計算"""
    K = scene.K
    errors_per_camera = defaultdict(list)
    all_errors = []
    
    for i, pt_idx in enumerate(sfm_result['point_indices']):
        X = sfm_result['points_3d'][i]
        X_h = np.append(X, 1)
        
        for cam_idx, (R, t) in sfm_result['cameras'].items():
            if not scene.visibility[cam_idx][pt_idx]:
                continue
            
            P = K @ np.hstack([R, t.reshape(-1, 1)])
            x_proj_h = P @ X_h
            x_proj = x_proj_h[:2] / x_proj_h[2]
            
            x_obs = scene.projections[cam_idx][pt_idx]
            error = np.linalg.norm(x_proj - x_obs)
            
            errors_per_camera[cam_idx].append(error)
            all_errors.append(error)
    
    return {
        'per_camera': {k: np.array(v) for k, v in errors_per_camera.items()},
        'all': np.array(all_errors)
    }

reproj_errors = compute_reprojection_error(sfm_result, scene)

print("=== 再投影誤差 ===")
print(f"全体:")
print(f"  平均: {np.mean(reproj_errors['all']):.2f} pixels")
print(f"  中央値: {np.median(reproj_errors['all']):.2f} pixels")
print(f"  最大: {np.max(reproj_errors['all']):.2f} pixels")

print(f"\nカメラ別:")
for cam_idx in sorted(reproj_errors['per_camera'].keys()):
    errors = reproj_errors['per_camera'][cam_idx]
    print(f"  Camera {cam_idx}: mean={np.mean(errors):.2f}, n={len(errors)}")

In [ ]:
def visualize_reprojection_errors(reproj_errors: Dict):
    """再投影誤差の可視化"""
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 全体のヒストグラム
    axes[0].hist(reproj_errors['all'], bins=30, edgecolor='black', alpha=0.7)
    axes[0].axvline(x=np.mean(reproj_errors['all']), color='red', linestyle='--',
                    label=f"Mean: {np.mean(reproj_errors['all']):.2f}px")
    axes[0].set_xlabel('Reprojection Error (pixels)', fontsize=12)
    axes[0].set_ylabel('Count', fontsize=12)
    axes[0].set_title('Distribution of Reprojection Errors', fontsize=12)
    axes[0].legend()
    
    # カメラ別の箱ひげ図
    camera_indices = sorted(reproj_errors['per_camera'].keys())
    data = [reproj_errors['per_camera'][i] for i in camera_indices]
    
    axes[1].boxplot(data, labels=[f'Cam {i}' for i in camera_indices])
    axes[1].set_xlabel('Camera', fontsize=12)
    axes[1].set_ylabel('Reprojection Error (pixels)', fontsize=12)
    axes[1].set_title('Reprojection Errors by Camera', fontsize=12)
    
    plt.tight_layout()
    plt.show()

visualize_reprojection_errors(reproj_errors)

---

## 9. まとめと次のステップ

### 学んだこと

1. **SfM の概要**: 画像からStructure（3D点群）とMotion（カメラ姿勢）を復元
2. **インクリメンタル SfM**: 2視点で初期化し、画像を1枚ずつ追加
3. **2視点初期化**: 本質行列 → カメラ姿勢 → 三角測量
4. **PnP 問題**: 既知の3D点から新しいカメラ姿勢を推定
5. **トラック**: マルチビューでの特徴点対応
6. **再投影誤差**: 復元品質の評価指標

### SfM パイプラインの要点

| ステップ | 入力 | 出力 | 主要アルゴリズム |
|----------|------|------|------------------|
| 特徴点検出 | 画像 | キーポイント | SIFT, ORB |
| マッチング | キーポイント | 対応点 | FLANN, Ratio test |
| 初期化 | 2画像の対応点 | 初期カメラ・3D点 | 8点法, RANSAC |
| カメラ追加 | 3D-2D対応 | 新カメラ姿勢 | PnP, RANSAC |
| 点追加 | 新カメラ + 既存カメラ | 新3D点 | 三角測量 |
| 最適化 | 全カメラ・全点 | 最適化された復元 | バンドル調整 |

### 次のノートブック

**60. バンドル調整**では：
- 再投影誤差の最小化
- スパース性の活用
- Levenberg-Marquardt法
- Ceres Solver の紹介

---

## 10. 自己評価クイズ

以下の質問に答えて理解度を確認しましょう：

1. Structure from Motion の「Structure」と「Motion」はそれぞれ何を指しますか？

2. インクリメンタル SfM と グローバル SfM の違いは？

3. 2視点初期化で本質行列から4つの解候補が出る理由と、正しい解を選ぶ方法は？

4. PnP 問題とは何ですか？なぜ SfM で重要ですか？

5. トラックとは何ですか？長いトラックが望ましい理由は？

6. 再投影誤差が大きい場合、考えられる原因は？

In [ ]:
# クイズの解答（隠し）
def show_quiz_answers():
    answers = """
    === 自己評価クイズ解答 ===
    
    1. Structure と Motion:
       - Structure: 3D点群（シーンの幾何学的構造）
       - Motion: カメラの姿勢（位置と向き）
       - 両者を同時に復元するのがSfM
    
    2. インクリメンタル vs グローバル:
       - インクリメンタル: 画像を1枚ずつ追加、累積的な誤差が問題
       - グローバル: 全画像の関係を同時に考慮、初期化が難しい
       - インクリメンタルは実装が容易で広く使われている
    
    3. 4つの解候補:
       - 本質行列のSVD分解での符号の不定性から4通りの(R, t)が出る
       - 正しい解を選ぶ: チェイラリティ条件（両カメラの前方に点がある）
       - 三角測量して両カメラでZ > 0の点が最も多い解を選択
    
    4. PnP 問題:
       - 既知の3D点と対応する2D画像点からカメラ姿勢を推定
       - SfMでの重要性: 新しい画像を追加する際に必要
       - 最低6点（DLT）または4点（P3P）で解ける
    
    5. トラック:
       - 同じ3D点に対応する複数画像での2D観測点の集合
       - 長いトラックが望ましい理由:
         - より多くの拘束条件を提供
         - バンドル調整での安定性向上
         - ドリフトの抑制
    
    6. 再投影誤差が大きい原因:
       - 特徴点マッチングの誤り（外れ値）
       - カメラキャリブレーションの誤差
       - レンズ歪みの補正不足
       - 動く物体（静的シーン仮定の違反）
       - バンドル調整の未実行または収束不足
    """
    print(answers)

# 解答を見るには以下のコメントを外して実行
# show_quiz_answers()

---

## ナビゲーション

- **前のノートブック**: [58. 特徴点検出とマッチング](58_feature_detection_matching_v1.ipynb)
- **次のノートブック**: [60. バンドル調整](60_bundle_adjustment_v1.ipynb)
- **カリキュラム**: [CURRICULUM_UNIT_0.3.md](CURRICULUM_UNIT_0.3.md)